In [1]:
# Leave lines below uncommented this if you get:
# OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torch.nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
from glomerulus import Glomerulus, Patch, KidneySampleDataset, generate_patches
from networks import CBAM_R2UNet_v2
from utils import read_tiff, dataset_label_mean
from losses import dice_loss

In [3]:
# train_image_names = [
#     'afa5e8098',
#     '4ef6695ce',
#     'c68fe75ea',
#     '26dc41664',
#     '095bf7a1f',
#     '54f2eec69',
#     '1e2425f28',
#     'e79de561c',
#     'cb2d976f4',
#     'b9a3865fc',
#     '8242609fa',
#     '0486052bb',
#     '2f6ecfcdf',
# ]

train_image_names = [
    'b2dc8411c',
    # 'aaa6a05cc'
]

val_image_names = [
    # 'b2dc8411c',
    'aaa6a05cc'
]

root_dir = './dataset/train'

patch_size = 512
num_train_samples = 1000
num_val_samples = 200

glomerulus_patch_ratio = 0.8
random_patch_ratio = 0.2

In [ ]:
patches = []
dataset_dir = './samples_dataset'

for ratio in range(0, 11):
    glomerulus_patch_ratio = np.round(ratio/10, 1)
    random_patch_ratio = np.round(1 - glomerulus_patch_ratio, 1)

    # Iterate through all images used for training
    for idx, image_name in enumerate(tqdm(train_image_names, desc='Loading Train Images')):
        image_path = os.path.join(root_dir, f'{image_name}.tiff')
        label_path = os.path.join(root_dir, f'{image_name}.json')
        try:
            # Sample patches from the image
            patches = generate_patches(
                patch_size = patch_size,
                num_samples = num_train_samples,
                image_path = image_path,
                label_path = label_path,
                # Dataset ratio config
                glomerulus_patch_ratio = glomerulus_patch_ratio,
                random_patch_ratio = random_patch_ratio,
                empty_patch = 0
            )
            
            # Create a dataset object from sampled patches and save as temp file (to save RAM when loading new imgs)
            dataset = KidneySampleDataset(patches)
            dataset.save(os.path.join(dataset_dir, f'train_chunk_{idx}.npy'))
        except:
            print('Error generating patches {image_path}')
            
    # Read saved temp dataset chunks and append as 1 single dataset
    train_dataset = KidneySampleDataset()
    for idx in tqdm(range(len(train_image_names)), desc='Appending Generated Datasets'):
        tmp_dataset = KidneySampleDataset()
        try:
            tmp_dataset.load(os.path.join(dataset_dir, f'train_chunk_{idx}.npy'))
            train_dataset.append_data(tmp_dataset)
        except:
            print(f'Error loading chunk {idx}')
    train_dataset.save(os.path.join(dataset_dir, f'train_{str(glomerulus_patch_ratio)}_{str(random_patch_ratio)}.npy'))

glomerulus_patch_ratio = 0
random_patch_ratio = 1
            
# Iterate through all images used for validating
for idx, image_name in enumerate(tqdm(val_image_names, desc='Loading Val Images')):
    image_path = os.path.join(root_dir, f'{image_name}.tiff')
    label_path = os.path.join(root_dir, f'{image_name}.json')
    try:
        # Sample patches from the image
        patches = generate_patches(
            patch_size = patch_size,
            num_samples = num_val_samples,
            image_path = image_path,
            label_path = label_path,
            # Dataset ratio config
            glomerulus_patch_ratio = glomerulus_patch_ratio,
            random_patch_ratio = random_patch_ratio,
            empty_patch = 0
        )
        
        # Create a dataset object from sampled patches and save as temp file (to save RAM when loading new imgs)
        dataset = KidneySampleDataset(patches)
        dataset.save(os.path.join(dataset_dir, f'val_chunk_{idx}.npy'))
    except:
        print('Error generating patches {image_path}')
        
# Read saved temp dataset chunks and append as 1 single dataset
val_dataset = KidneySampleDataset()
for idx in tqdm(range(len(val_image_names)), desc='Appending Generated Datasets'):
    tmp_dataset = KidneySampleDataset()
    try:
        tmp_dataset.load(os.path.join(dataset_dir, f'val_chunk_{idx}.npy'))
        val_dataset.append_data(tmp_dataset)
    except:
        print(f'Error loading chunk {idx}')
val_dataset.save(os.path.join(dataset_dir, f'val.npy'))

Loading Train Images:   0%|          | 0/1 [00:00<?, ?it/s]

/home/patipol/Documents/automated-visual-acuity-tester/speech-model/whisper_env/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Rendering Patches:   0%|          | 0/1000 [00:00<?, ?it/s]

Appending Generated Datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Loading Train Images:   0%|          | 0/1 [00:00<?, ?it/s]

Rendering Patches:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
dataset[0]